# Examples with run file and C++

This notebooks shows how to **write and execute run files** and **call C++ functions** using the interface from **consav**.

**Requirements:** You must have these two compilers installed:

* **vs**: Free *Microsoft Visual Studio 2017 Community Edition* ([link](https://visualstudio.microsoft.com/downloads/))
* **intel:** Costly *Intel Parallel Studio 2018 Composer Edition* ([link](https://software.intel.com/en-us/parallel-studio-xe))

**Computer used for timings:** Windows 10 computer with two Intel(R) Xeon(R) Gold 6154 3.00 GHz CPUs (18 cores, 36 logical processes each) and 192 GB of RAM.

# Setup

In [1]:
# reload module each time cell is run
%load_ext autoreload
%autoreload 2

# load the BufferStockModel module
from BufferStockModel import BufferStockModelClass

# Run files

You can create a run file (here **run.py**) calling a specific method (here **test()**), and then run it in a system command.

In [2]:
# model = BufferStockModelClass(name='',solmethod='egm')
# model.write_run_file(filename='run.py',method='test') # open run.py and have a look
# !python run.py

This is e.g. beneficial when testing how much speed-up numba provides:

In [3]:
# from consav import runtools
# model.write_run_file(filename='run.py',method='test',solmethod='egm',Np=2000,Nm=2000,Na=2000)
# # here we are making the grids more dense to increase the runtime

# for threads in [1,4,8,16,32,64]:
#     runtools.write_numba_config(threads=threads,threading_layer='omp')
#     print(f'threads = {threads}')
#     !python run.py
#     print('')

In [4]:
# for threads in [1,4,8,16,32,64]:
#     runtools.write_numba_config(threads=threads,threading_layer='tbb')
#     print(f'threads = {threads}')
#     !python run.py
#     print('')

## C++

**Note:** The code below is only for **Windows** and requires installing one of the following two compilers:

* **vs**: Free *Microsoft Visual Studio 2017 Community Edition* ([link](https://visualstudio.microsoft.com/downloads/))
* **intel:** Costly *Intel Parallel Studio 2018 Composer Edition* ([link](https://software.intel.com/en-us/parallel-studio-xe))

**Troubleshooting:** The installation paths for the compiler might need to be adjusted. These are passed as argument to the cpptools.compile() function. Defaults are specified in model.vs_path, model.intel_path, and model.intel_vs_version.

*Additional details are provided in the notebook **Calling C++** in the **Numba and C++** folder.*

### Overview

The **ModelClass** class also have an interface to C++. 

All C++ files should be in **cpp_funcs/**. The main file (here **EGM.cpp**) should:

1. Include an **export macro**

  ```
  #define EXPORT extern "C" __declspec(dllexport)
  ```


2. Include the following **struct definitions**:

  ```
  #include "par_struct.cpp"
  #include "sol_struct.cpp"
  #include "sim_struct.cpp"  
  ```
  
  
3. Define **gateway functions** as:

  ```
  EXPORT void myfunction(par_struct *par, sol_struct *sol, sim_struct *sim)
  ```


4. If using visual studio the following function should be included:

  ```
  EXPORT void setup_omp(){
      SetEnvironmentVariable("OMP_WAIT_POLICY", "passive"); 
  }
  ```   

Note, that the structure is such that all C++ functions for simplicity are taking both par, sol and sim as input. The numba types automatically infered and saved `.parlist`, `.sollist` and `.simlist` are used for writing `par_struct.cpp`, `sol_struc.cpp`, and `sim_struct.cpp`.

### Linking

When initializing a model the **compiler option** can be chosen (defaults to _vs_).

The most import C++ methods are:

1. **setup_cpp()**: Writes *par\_struct.cpp*, *sol\_struct.cpp*, *sim\_strunct.cpp* in cppfuncs/.
2. **link_cpp(FILE,FUNCS)**: Link to C++ file **cppfuncs/FILE.cpp** with list of functions in **FUNCS**.
3. **call_cpp(FILE,FUNC)**: Call **FUNC** from C++ **FILE**.
4. **delink_cpp(FILE)**: Delink C++ library.

### Example

In [5]:
for compiler in ['vs','intel']:
    for threads in [1,4,8,16,32,64]:
        model = BufferStockModelClass(name='',solmethod='egm',Np=2000,Nm=2000,Na=2000)
        model.cppinfo['compiler'] = compiler
        print(f'compiler = {compiler}, threads = {threads},')
        model.par.cppthreads = threads
        tic,toc = model.solve_cpp()
        print(f'solved in {toc-tic:.1f} secs')
        model.checksum()
        print('')

AssertionError: T is <class 'int'>, not float, but not on the list

### Installation paths

In [ ]:
model = BufferStockModelClass(name='')
print(model.cppinfo['vs_path'])
print(model.cppinfo['intel_path'])
print(model.cppinfo['intel_vs_version'])